## Ejemplo 3.3.1

Este apunte computa el ejemplo 3.3.1 de @daertrycke_risk_2017 donde se tiene un productor neutral al riesgo que resuelve

$$\min_{x,Y}I\cdot x+\tau\mathbb{E}_{\Theta}\left[(C-P_\omega)\cdot Y_\omega\right]\quad\text{subject to}\quad Y_\omega\in[0,x]$$

y la demanda como resultado de resolver

$$\min_Q\tau\mathbb{E}_\Theta\left[P_\omega\cdot Q_\omega-U_\omega(Q_\omega)\right]$$

donde $U_\omega(Q_\omega)=A_\omega\cdot Q_\omega -\frac{B}{2}\cdot Q^2_\omega$

Quedando la función como:

$$
\min_{Q,x} Ix+\tau E_{\Theta}\left[CY_w-A_wQ_w+\frac{B}{2}Q_w^2\right] \quad \forall w
$$

Sujeto a: $$
 Q_w=Y_w
 $$

$$
0\leq Y_w \leq x
$$


In [ ]:
#| eval: false
#| echo: false

# Instalamos paquetes necesarios (sólo primera vez)
import Pkg
Pkg.add("Gurobi")
Pkg.add("JuMP")

### Parametrización


In [ ]:
#| echo: false
using JuMP,Gurobi,DataFrames
A=[300,350,400,450,500];
θ=[0.2,0.2,0.2,0.2,0.2];
B=1;
I=90;
C=60;
tau=8760;
# Crear un DataFrame
df = DataFrame(Variable=["A", "θ"],
               Escenario1=[A[1], θ[1]],
               Escenario2=[A[2], θ[2]],
               Escenario3=[A[3], θ[3]],
               Escenario4=[A[4], θ[4]],
               Escenario5=[A[5], θ[5]])

# Mostrar el DataFrame
println(df)

## Transición a nueva energía

Se tiene ahora el modelo adaptado hacia la transición energética. $$
\min_{Q,Y,x1,x2,T} I_1(x_1) + I_2(x_2) + \tau E_{\Theta}\left[(C+CT_w - S_w)Y_w - A_wQ_w + \frac{B}{2}Q_w^2 \right]  \quad
$$

Sujeto a las siguientes restricciónes: $$
Q_w = Y_w \quad \forall w
$$ $$
0 \leq Y_w \leq x_1 + x_2 \quad \forall w
$$ $$
0 \leq T_w \leq T_{w+1} \quad \text{con } w = 1, 2, \ldots, 5
$$ $$
0 \leq \alpha \leq 1 \quad
$$


In [ ]:
# Parámetros iniciales
A = [300, 350, 400, 450, 500]
theta = [0.2, 0.2, 0.2, 0.2, 0.2]
s = 80
c = 60
alpha = 0.7

In [ ]:
model = Model(Gurobi.Optimizer)

set_optimizer_attribute(model, "NonConvex", 2)

# Variables
@variable(model, x >= 0)  # Capacidad
@variable(model, Y[i in 1:5] >= 0) # Producción en escenario w
@variable(model, Q[i in 1:5] >= 0) # Demanda en escenario w
@variable(model, 0 <= T[i in 1:5] <= 1) # Transición en escenario w

# Restricciones
@constraint(model, demanda[i in 1:5], Y[i] == Q[i])
@constraint(model, capacidad[i in 1:5], Y[i] - x <= 0.0)
@constraint(model, transicion[i in 2:5], T[i] >= T[i-1])
@constraint(model, incremento[i in 2:5], T[i] - T[i-1] <= 0.2)
@constraint(model, T[1] == 0)



S = [s * T[i] for i in 1:5]
CT = [c * T[i] * (1 - alpha) for i in 1:5]

# Función objetivo
@objective(model, Min, I*1000*x + tau*(sum(theta[i]*( (C + CT[i] - S[i]) * Y[i] - A[i]*Q[i] + B*0.5*(Q[i]^2)) for i in 1:5)))

# Resolver
JuMP.optimize!(model)

# Resultados
println("Q = ", JuMP.value.(Q))
println("T = ", JuMP.value.(T))
println("x = ", JuMP.value(x))

Los parámetros de demanda para cada escenario y su probabilidad están fijados como:


In [ ]:
model=Model(Gurobi.Optimizer)

#Variables
@variable(model, x>=0)  #Capacidad
@variable(model, Y[i in 1:5]>=0) # Produccion en escenario w
@variable(model,Q[i in 1:5]>=0) # Demanda en escenario w

#Restricciones
@constraint(model, demanda[i in 1:5], Y[i] == Q[i] )
@constraint(model, capacidad[i in 1:5], Y[i]-x<=0.0)

#FO
@objective(model, Min, I*1000*x + tau*(sum(theta[i]*(C*Y[i] - A[i]*Q[i] + B*0.5*(Q[i]^2)) for i in 1:5) ))

#Resolver
JuMP.optimize!(model)

In [ ]:
println("Q = ", JuMP.value.(Q))
Qlist=JuMP.value.(Q)

In [ ]:
#| eval: false
j=1
P=[]
while j<6
    Pi=A[j]-B*Qlist[j]
    append!(P, Pi)
    j=j+1
end
println("P = ",P)
println("x = ", JuMP.value(x))